<a href="https://colab.research.google.com/github/gilbertcarey/skyspec/blob/main/Skyspecs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

---



In [2]:
!pip install geopandas 2>/dev/null 1>&2
!pip install plotly-geo 2>/dev/null 1>&2


In [3]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
import os
import time
from geopy.distance import distance
import folium
import random
from pprint import pprint
import pickle
import geopandas as gpd
import branca
folium.__version__,
from os import access, R_OK
from os.path import isfile
from branca.element import MacroElement
from jinja2 import Template
import math
import folium
from shapely.geometry import Point
from folium.map import LayerControl
import plotly.express as px
pd.set_option('display.max_columns', None)

# Load data

In [4]:
os.chdir("/content/drive/MyDrive/Skyspecs")
os.listdir()

['uswtdb_v4_3_20220114.geojson',
 'Brief.pdf',
 'uswtdb.csv',
 'georef-united-states-of-america-county.geojson',
 'lightning_by_state.xlsx',
 'Copy of Skyspecs.ipynb',
 'Skyspecs.ipynb']

# Get data

In [5]:
lightning_by_state_df = pd.read_excel('lightning_by_state.xlsx')
uswtd_df = pd.read_csv('uswtdb.csv', encoding='ISO-8859-1')


## lightning data

In [6]:
lightning_by_state_df.sample(5)

,StateCode,State,Total Lightning in 2019,Average Total Lightning 2015 to 2019,Average Lightning Count Per Square Mile 2015 to 2019
11,HI,Hawaii,11893,34560,3.2
49,WI,Wisconsin,2081049,1922736,34.4
43,TX,Texas,47397975,41083472,154.9
28,NV,Nevada,721732,819506,7.4
0,AL,Alabama,3970233,5773460,110.7


In [7]:
lightning_by_state_df.describe()

,Total Lightning in 2019,Average Total Lightning 2015 to 2019,Average Lightning Count Per Square Mile 2015 to 2019
count,5.100000e+01,5.100000e+01,51.000000
mean,4.380080e+06,4.259427e+06,68.066667
std,7.235071e+06,6.436253e+06,61.799546
min,8.005000e+03,7.195000e+03,0.300000
25%,4.351445e+05,4.154940e+05,13.800000
50%,2.867912e+06,2.784901e+06,47.700000
75%,5.038338e+06,5.588456e+06,105.550000
max,4.739798e+07,4.108347e+07,244.100000


In [8]:
lightning_by_state_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   StateCode                                             51 non-null     object 
 1   State                                                 51 non-null     object 
 2   Total Lightning in 2019                               51 non-null     int64  
 3   Average Total Lightning 2015 to 2019                  51 non-null     int64  
 4   Average Lightning Count Per Square Mile 2015 to 2019  51 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.1+ KB


## turbine data

In [9]:
uswtd_df.sample(5)

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat
30284,3111358,29-072903,2019-WTE-3867-OE,NaN,62475.0,MO,Lawrence County,29109,Kings Point,2021.0,69,149.40,Vestas,V110-2.0,2000.0,120.0,110.0,9503.32,175.0,0,NaN,3,3,8/23/2021,Digital Globe,-93.953308,37.283310
46289,3025658,19-020766,2008-ACE-1911-OE,20656.0,56797.0,IA,Mitchell County,19131,Pioneer Prairie II,2009.0,43,70.95,Vestas,V82-1.65,1650.0,80.0,82.0,5281.02,121.0,0,NaN,3,3,11/19/2017,Digital Globe,-92.727196,43.464790
30550,3023134,41-020183,2017-WTW-5950-OE,373.0,56468.0,OR,Sherman County,41055,Klondike IIIA,2008.0,51,76.50,GE Wind,GE1.5-77,1500.0,80.0,77.0,4656.63,118.6,0,NaN,3,3,6/29/2019,Digital Globe,-120.506783,45.559593
25075,3097783,48-163585,2018-WTW-13512-OE,NaN,62562.0,TX,Upton County,48461,High Lonesome,2019.0,142,449.94,Nordex,AW132/3465,3465.0,85.0,132.0,13684.78,150.9,0,NaN,3,3,12/1/2019,Digital Globe,-101.926788,31.146738
48734,3091539,48-154710,2018-WTW-12019-OE,NaN,62259.0,TX,Crockett County,48105,Ranchero,2019.0,120,300.00,GE Wind,GE2.5-127,2500.0,89.0,127.0,12667.69,152.4,0,NaN,3,3,7/26/2019,Digital Globe,-101.682190,31.014912


In [10]:
uswtd_df.describe()

,case_id,usgs_pr_id,eia_id,t_fips,p_year,p_tnum,p_cap,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit,retrofit_year,t_conf_atr,t_conf_loc,xlong,ylat
count,7.080800e+04,38263.000000,65015.000000,70808.000000,70195.000000,70808.000000,66326.000000,65328.000000,64628.000000,64874.000000,64874.000000,64628.000000,70808.000000,5986.000000,70808.000000,70808.000000,70808.000000,70808.000000
mean,3.058490e+06,27523.587225,57877.751642,32244.494097,2011.650659,104.360552,170.176128,1963.534151,81.054571,95.663717,7618.500657,129.050593,0.084538,2018.640327,2.766509,2.883784,-100.086457,38.479503
std,3.242377e+04,13564.446593,5984.516487,15498.246596,7.879099,93.954786,104.478693,717.067912,12.028726,23.424346,3309.229282,22.185981,0.278196,1.112294,0.597195,0.462326,11.141025,5.429730
min,3.000001e+06,1.000000,90.000000,2013.000000,1981.000000,1.000000,0.050000,50.000000,19.000000,13.400000,141.030000,30.400000,0.000000,2015.000000,1.000000,1.000000,-171.713074,13.389381
25%,3.032230e+06,18625.500000,56763.000000,19081.000000,2008.000000,56.000000,99.000000,1500.000000,80.000000,82.000000,5281.020000,121.000000,0.000000,2018.000000,3.000000,3.000000,-103.037155,34.428043
50%,3.050978e+06,28598.000000,57752.000000,35057.000000,2012.000000,85.000000,158.000000,2000.000000,80.000000,100.000000,7853.980000,130.100000,0.000000,2019.000000,3.000000,3.000000,-99.393761,39.052442
75%,3.090448e+06,38719.500000,60338.000000,48141.000000,2018.000000,121.000000,211.220000,2300.000000,87.000000,110.000000,9503.320000,145.100000,0.000000,2020.000000,3.000000,3.000000,-95.202499,42.812810
max,3.118671e+06,49135.000000,65270.000000,72133.000000,2021.000000,731.000000,525.020000,6000.000000,131.000000,155.000000,18869.190000,199.600000,1.000000,2020.000000,3.000000,3.000000,144.722656,66.839905


In [11]:
uswtd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70808 entries, 0 to 70807
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   case_id        70808 non-null  int64  
 1   faa_ors        65016 non-null  object 
 2   faa_asn        65686 non-null  object 
 3   usgs_pr_id     38263 non-null  float64
 4   eia_id         65015 non-null  float64
 5   t_state        70808 non-null  object 
 6   t_county       70808 non-null  object 
 7   t_fips         70808 non-null  int64  
 8   p_name         70808 non-null  object 
 9   p_year         70195 non-null  float64
 10  p_tnum         70808 non-null  int64  
 11  p_cap          66326 non-null  float64
 12  t_manu         65168 non-null  object 
 13  t_model        65031 non-null  object 
 14  t_cap          65328 non-null  float64
 15  t_hh           64628 non-null  float64
 16  t_rd           64874 non-null  float64
 17  t_rsa          64874 non-null  float64
 18  t_ttlh

# Summary

|No|Field|NaN count| Type| Impute Action| Commentary|
|---|---|---|---|---|---|
|0|case_id|0|int64||Unique identifier, specific to turbine|
|1|faa_ors|5792|object|Drop for now |Might be interesting to replace missing t_hh, t_rd or combination of both. Could impute NaNs using links to FAA obstactle files www.aeronav.faa.gov/Obst_Data/.|
|2|faa_asn|5122|object|Drop for now|unique identifier for obstruction evaluation report. Could impute NaNs using this sourceThese reports contain contact information for each item. Might be useful.|
|3|usgs_pr_id|32545|float64|Drop for now|unique identifier to USGS data, contains similar data to this dataset. Might be used to replace nans www.sciencebase.gov/catalog/item/55c4aa9be4b033ef52106e2f
|4|eia_id|5793|float64|Drop for now|https://www.eia.gov/renewable/
|5|t_state|0|object||State|
|6|t_county|0|object||County|
|7|t_fips|0|int64||Location parameter unique to each county|
|8|p_name|0|object||Name of windpower project|
|9|p_year|613|float64|Impute using project mean year, or 0 if no project year info|Year that the turbines became operational|
|10|p_tnum|0|int64||Number of turbines in the project|
|11|p_cap|4482|float64||Cumalative capacity of the project (MW)|
|12|t_manu|5640|object||Equipment manufacture date|
|13|t_model|5777|object||Turbine model|
|14|t_cap|5480|float64||Power output of the turbine - rated capacity (KW)|
|15|t_hh|6180|float64||Turbine hub height in meters (m)|
|16|t_rd|5934|float64||Turbine rotor diameter in meters (m)|
|17|t_rsa|5934|float64||Turbine rotor swept area in square meters (m2)|
|18|t_ttlh|6180|float64||Turbine total height from ground to tip of a blade at its apex in meters (m)|
|19|retrofit|0|int64|Drop, we can use retrofit_year only|0, no known retrofit; 1, a retrofit has happened|
|20|retrofit_year|64822|float64|0 if unknown refit, otherwise year|If retrofit, then this has the year|
|21|t_conf_atr|0|int64||Confidence level for turbine attrs, 1-3, 1 lowest, 3 highest|
|22|t_conf_loc|0|int64||Confidence level for turbine loc, 1-3 as above, depends on image|
|23|t_img_date|8316|object||image date, NAIP specifies year only|
|24|t_img_srce|0|object||Source of image to verify location|
|25|xlong|0|float64|Longitude|
|26|ylat|0|float64||Latitude|

We have 70,808 data points (turbines), each with 27 features.
The case_id feature is unique.

# Exploration

## References to other systems 

faa_ors, faa_asn, usgs_pr_id, eia_id are not unique - references to other databases. We can drop for the moment

In [12]:
uswtd_df.faa_ors.value_counts().max(), uswtd_df.faa_asn.value_counts().max(), uswtd_df.usgs_pr_id.value_counts().max(), uswtd_df.eia_id.value_counts().max()

(2, 62, 2, 432)

In [13]:
# Expore duplicates for identifiers to other systems
fields = ['faa_ors', 'faa_asn', 'usgs_pr_id', 'eia_id']
for field in fields:
  #field = 'eia_id'
  tmp = uswtd_df[uswtd_df[field].notna()]
  tmp[tmp[field].sort_values().duplicated(keep=False)].sort_values(by=field)
  print(field)
  #print(tmp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


faa_ors
faa_asn
usgs_pr_id
eia_id


Drop the columns

In [ ]:
if 'faa_ors' in uswtd_df.columns:
  uswtd_df.drop(columns=['faa_ors','faa_asn','usgs_pr_id', 'eia_id'], inplace=True)

Zip up latitude and longgitude columns into one location field per case id.\
Note that location is unique for each case id

In [ ]:
uswtd_df['loc']=list(zip(uswtd_df['xlong'], uswtd_df['ylat']))
uswtd_df['loc'].duplicated().sum()

0

## retrofit, retrofit_year

We can see that retrofit_year is NaN when refit = 0 -> what we might expect if there has been no refit.

We can drop retrofit if we set retrofit_year to 0 for no refit.

In [ ]:

print('Number of retrofit_year NaNs when retrofit=0: ', uswtd_df[uswtd_df.retrofit==0]['retrofit_year'].isna().sum())
print('Total number of retrofit_year = NaNs (ignore retrofit): ', uswtd_df['retrofit_year'].isna().sum())
# It looks like we only have to deal with retrofit==0

Number of retrofit_year NaNs when retrofit=0:  64822
Total number of retrofit_year = NaNs (ignore retrofit):  64822


Substitue 0 for NaNs for retrofit_year

In [ ]:
if uswtd_df['retrofit_year'].isna().sum() > 0:
  #uswtd_df['retrofit_year'] = uswtd_df['retrofit_year'].map(lambda x: str(int(x)) if not math.isnan(x) else 'Not Refitted')
  uswtd_df['retrofit_year'] = uswtd_df['retrofit_year'].map(lambda x: int(x) if not math.isnan(x) else 0)

In [ ]:
uswtd_df['retrofit_year'].value_counts()

0       64822
2019     1893
2020     1642
2017     1317
2018     1125
2015        9
Name: retrofit_year, dtype: int64

Look at 2019 just to cross check what the data looks like

In [ ]:
uswtd_df[uswtd_df['retrofit_year']==2019]

,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,loc
1860,3104449,ND,Barnes County,38003,Ashtabula Wind Center,2008.0,99,148.5,GE Wind,GE1.5-91,1500.0,80.0,91.0,6503.88,125.6,1,2019,3,3,1/1/2012,NAIP,-97.916191,47.131992,"(-97.916191, 47.131992)"
1861,3104399,ND,Barnes County,38003,Ashtabula Wind Center,2008.0,99,148.5,GE Wind,GE1.5-91,1500.0,80.0,91.0,6503.88,125.6,1,2019,3,3,1/1/2012,NAIP,-97.905594,47.113293,"(-97.905594, 47.113293)"
1862,3104440,ND,Barnes County,38003,Ashtabula Wind Center,2008.0,99,148.5,GE Wind,GE1.5-91,1500.0,80.0,91.0,6503.88,125.6,1,2019,3,3,1/1/2012,NAIP,-97.913589,47.133392,"(-97.913589, 47.133392)"
1863,3104412,ND,Barnes County,38003,Ashtabula Wind Center,2008.0,99,148.5,GE Wind,GE1.5-91,1500.0,80.0,91.0,6503.88,125.6,1,2019,3,3,1/1/2012,NAIP,-97.894295,47.161991,"(-97.894295, 47.161991)"
1864,3104401,ND,Barnes County,38003,Ashtabula Wind Center,2008.0,99,148.5,GE Wind,GE1.5-91,1500.0,80.0,91.0,6503.88,125.6,1,2019,3,3,1/1/2012,NAIP,-97.928291,47.129791,"(-97.928291, 47.129791)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65520,3104835,ND,Burleigh County,38015,Wilton Wind Energy Center II,2009.0,33,49.5,GE Wind,GE1.5-91,1500.0,80.0,91.0,6503.88,125.6,1,2019,3,3,1/1/2012,NAIP,-100.755592,47.109592,"(-100.755592, 47.109592)"
65521,3104816,ND,Burleigh County,38015,Wilton Wind Energy Center II,2009.0,33,49.5,GE Wind,GE1.5-91,1500.0,80.0,91.0,6503.88,125.6,1,2019,3,3,1/1/2012,NAIP,-100.662788,47.131092,"(-100.662788, 47.131092)"
65522,3104809,ND,Burleigh County,38015,Wilton Wind Energy Center II,2009.0,33,49.5,GE Wind,GE1.5-91,1500.0,80.0,91.0,6503.88,125.6,1,2019,3,3,1/1/2012,NAIP,-100.661186,47.125893,"(-100.661186, 47.125893)"
65523,3104817,ND,Burleigh County,38015,Wilton Wind Energy Center II,2009.0,33,49.5,GE Wind,GE1.5-91,1500.0,80.0,91.0,6503.88,125.6,1,2019,3,3,1/1/2012,NAIP,-100.630287,47.114491,"(-100.630287, 47.114491)"


Drop retrofit as we have that data in retrofit year

In [ ]:
if 'retrofit' in uswtd_df.columns:
  del uswtd_df['retrofit']

## Look at p_year



replace NaN with average of year for every p_name, if we have no date information, we can use 0


In [ ]:
uswtd_df['p_year'].isna().sum(0)

613

In [ ]:
uswtd_df['p_year'] = uswtd_df['p_year'].fillna(uswtd_df.groupby('p_name')['p_year'].transform('mean'))
uswtd_df['p_year'] = uswtd_df['p_year'].fillna(uswtd_df.groupby('p_name')['p_year'].transform(lambda x:0)).astype(int)

In [ ]:
uswtd_df[uswtd_df['p_year'].isna()]

,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,loc


## p_cap, p_tnum, t_cap


p_tnum has no missing values

### p_cap
Should match p_cap(MW) = p_tnum * t_cap(KW) / 1000

In [ ]:
uswtd_df['p_capacity_check'] = uswtd_df['p_tnum'] * uswtd_df['t_cap'] /1000
uswtd_df[['p_cap', 'p_tnum', 't_cap', 'p_capacity_check']].sample(5) # looks like the calculation is correct!

,p_cap,p_tnum,t_cap,p_capacity_check
64955,180.08,52,4800.0,249.6
38626,84.75,297,NaN,NaN
26472,230.00,100,2300.0,230.0
4915,198.00,132,1500.0,198.0
45763,175.10,103,1700.0,175.1


In [ ]:
del uswtd_df['p_capacity_check']

Look for cases where p_cap is NaN but p_tnum and t_cap are not NaN (so we can calculate p_cap)

In [ ]:
uswtd_df[(uswtd_df['p_tnum'].notna())&
         (uswtd_df['t_cap'].notna())&
         (uswtd_df['p_cap'].isna())][['p_name', 'p_tnum', 't_cap','p_cap']]

,p_name,p_tnum,t_cap,p_cap


Lets find the situation where we have no turbine model, manufacturer and no p_cap

In [ ]:
uswtd_df[(uswtd_df['p_cap'].isna()) & (uswtd_df['t_model'].isna()) & (uswtd_df['t_manu'].isna())]

,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,loc
29056,3001878,MI,Kalamazoo County,26077,Kalamazoo Valley Community College,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3,4/29/2012,Digital Globe,-85.683670,42.225143,"(-85.68367, 42.225143)"
30672,3000932,IL,Coles County,17029,LLC 1,2011,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3,10/27/2018,Digital Globe,-88.387367,39.417534,"(-88.387367, 39.417534)"
31193,3006675,WI,Manitowoc County,55071,Lakeshore Technical College,2004,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3,NaN,Bing Maps Aerial,-87.755493,43.928093,"(-87.755493, 43.928093)"
53132,3064958,NM,Santa Fe County,35049,Santa Fe Public Works,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3,7/10/2016,Digital Globe,-106.069237,35.641449,"(-106.069237, 35.641449)"
62057,3015261,HI,Maui County,15009,University of Hawaii at Maui,2012,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3,5/8/2018,Digital Globe,-156.481689,20.891296,"(-156.481689, 20.891296)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70803,3053232,NY,Wyoming County,36121,unknown Wyoming County,2016,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3,5/20/2017,Digital Globe,-78.187935,42.740818,"(-78.187935, 42.740818)"
70804,3101958,TX,Young County,48503,unknown Young County,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,8/2/2020,Digital Globe,-98.907600,33.149414,"(-98.9076, 33.149414)"
70805,3040944,TX,Young County,48503,unknown Young County 1,2011,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3,7/16/2018,Digital Globe,-98.551094,33.093292,"(-98.551094, 33.093292)"
70806,3055918,CO,Yuma County,8125,unknown Yuma County,2016,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3,5/17/2017,Digital Globe,-102.716949,40.037548,"(-102.716949, 40.037548)"


### t_cap

Lets calculate t_cap where we have p_cap and p_tnum

p_cap(MW) = p_tnum * t_cap(KW) / 1000

=> t_cap(KW) = 1000 * p_cap(MW) / p_tnum


In [ ]:
uswtd_df[(uswtd_df['p_tnum'].notna())&
         (uswtd_df['t_cap'].isna())&
         (uswtd_df['p_cap'].notna())][['p_name', 'p_tnum', 't_cap','p_cap']]

,p_name,p_tnum,t_cap,p_cap
799,Altech III,201,NaN,21.71
800,Altech III,201,NaN,21.71
801,Altech III,201,NaN,21.71
802,Altech III,201,NaN,21.71
803,Altech III,201,NaN,21.71
...,...,...,...,...
69938,unknown Tehachapi Wind Resource Area 3,59,NaN,35.40
69939,unknown Tehachapi Wind Resource Area 3,59,NaN,35.40
69940,unknown Tehachapi Wind Resource Area 3,59,NaN,35.40
69941,unknown Tehachapi Wind Resource Area 3,59,NaN,35.40


1000 * p_cap(MW) / p_tnum

In [ ]:
p_cap = uswtd_df.columns.get_loc('p_cap')
p_tnum = uswtd_df.columns.get_loc('p_tnum')
uswtd_df.loc[(uswtd_df['p_tnum'].notna())&
          (uswtd_df['t_cap'].isna())&
          (uswtd_df['p_cap'].notna()),['t_cap']] = uswtd_df[(uswtd_df['p_tnum'].notna())&
                                                  (uswtd_df['t_cap'].isna())&
                                                  (uswtd_df['p_cap'].notna())].apply(lambda row:1000 * p_cap/p_tnum, axis=1)


 If we can not calculate, lets put zero in for t_cap - we can come back around and look for the make/model and infer from that

In [ ]:
uswtd_df['t_cap'].fillna(0, inplace=True)

In [ ]:
uswtd_df.t_cap.isna().sum()

0

### p_cap again
Replace remaining NaNs with 0 as we dont know.


In [ ]:
uswtd_df['p_cap'].fillna(0, inplace=True)

In [ ]:
uswtd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70808 entries, 0 to 70807
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   case_id        70808 non-null  int64  
 1   t_state        70808 non-null  object 
 2   t_county       70808 non-null  object 
 3   t_fips         70808 non-null  int64  
 4   p_name         70808 non-null  object 
 5   p_year         70808 non-null  int64  
 6   p_tnum         70808 non-null  int64  
 7   p_cap          70808 non-null  float64
 8   t_manu         65168 non-null  object 
 9   t_model        65031 non-null  object 
 10  t_cap          70808 non-null  float64
 11  t_hh           64628 non-null  float64
 12  t_rd           64874 non-null  float64
 13  t_rsa          64874 non-null  float64
 14  t_ttlh         64628 non-null  float64
 15  retrofit_year  70808 non-null  int64  
 16  t_conf_atr     70808 non-null  int64  
 17  t_conf_loc     70808 non-null  int64  
 18  t_img_

In [ ]:
uswtd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70808 entries, 0 to 70807
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   case_id        70808 non-null  int64  
 1   t_state        70808 non-null  object 
 2   t_county       70808 non-null  object 
 3   t_fips         70808 non-null  int64  
 4   p_name         70808 non-null  object 
 5   p_year         70808 non-null  int64  
 6   p_tnum         70808 non-null  int64  
 7   p_cap          70808 non-null  float64
 8   t_manu         65168 non-null  object 
 9   t_model        65031 non-null  object 
 10  t_cap          70808 non-null  float64
 11  t_hh           64628 non-null  float64
 12  t_rd           64874 non-null  float64
 13  t_rsa          64874 non-null  float64
 14  t_ttlh         64628 non-null  float64
 15  retrofit_year  70808 non-null  int64  
 16  t_conf_atr     70808 non-null  int64  
 17  t_conf_loc     70808 non-null  int64  
 18  t_img_

## t_manu, t_model

Count NaNs for model and manufacturer. We could do more here and look at the dimensions of turbines and try and figure out the manufacturer and model - maybe on the next pass

In [ ]:
uswtd_df[uswtd_df['t_manu'].isna()][['t_manu', 't_model']].isna().sum()

t_manu     5640
t_model    5630
dtype: int64

In [ ]:
uswtd_df['t_manu'].fillna('Unknown', inplace=True)
uswtd_df['t_model'].fillna('Unknown', inplace=True)

## t_hh, t_rd, t_rsa

In [ ]:
uswtd_df[['t_hh','t_rd', 't_rsa']]

,t_hh,t_rd,t_rsa
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,87.5,125.0,12271.85
4,87.5,125.0,12271.85
...,...,...,...
70803,NaN,NaN,NaN
70804,NaN,NaN,NaN
70805,NaN,NaN,NaN
70806,NaN,NaN,NaN


In [ ]:
uswtd_df[(uswtd_df['t_model'].isna())&(uswtd_df['t_manu'].isna() )][['p_year','t_manu', 't_model', 't_cap', 't_ttlh','t_hh']]

,p_year,t_manu,t_model,t_cap,t_ttlh,t_hh


In [ ]:
uswtd_df[['p_year','t_manu', 't_model']]

,p_year,t_manu,t_model
0,1987,Vestas,Unknown
1,1987,Vestas,Unknown
2,1987,Vestas,Unknown
3,2017,Nordex,AW125/3000
4,2017,Nordex,AW125/3000
...,...,...,...
70803,2016,Unknown,Unknown
70804,0,Unknown,Unknown
70805,2011,Unknown,Unknown
70806,2016,Unknown,Unknown


In [ ]:
uswtd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70808 entries, 0 to 70807
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   case_id        70808 non-null  int64  
 1   t_state        70808 non-null  object 
 2   t_county       70808 non-null  object 
 3   t_fips         70808 non-null  int64  
 4   p_name         70808 non-null  object 
 5   p_year         70808 non-null  int64  
 6   p_tnum         70808 non-null  int64  
 7   p_cap          70808 non-null  float64
 8   t_manu         70808 non-null  object 
 9   t_model        70808 non-null  object 
 10  t_cap          70808 non-null  float64
 11  t_hh           64628 non-null  float64
 12  t_rd           64874 non-null  float64
 13  t_rsa          64874 non-null  float64
 14  t_ttlh         64628 non-null  float64
 15  retrofit_year  70808 non-null  int64  
 16  t_conf_atr     70808 non-null  int64  
 17  t_conf_loc     70808 non-null  int64  
 18  t_img_

## t_img_date

In [ ]:
#uswtd_df[uswtd_df['t_img_date'].isna()][['t_img_date', 't_img_srce']]
uswtd_df[uswtd_df['t_img_date'].isna()][ 't_img_srce'].value_counts()

Bing Maps Aerial    8316
Name: t_img_srce, dtype: int64

In [ ]:
uswtd_df[uswtd_df['t_img_date'].notna()][['t_img_date', 't_img_srce']]

,t_img_date,t_img_srce
0,5/8/2018,Digital Globe
1,5/8/2018,Digital Globe
2,5/8/2018,Digital Globe
3,4/24/2017,Digital Globe
4,6/1/2017,Digital Globe
...,...,...
70803,5/20/2017,Digital Globe
70804,8/2/2020,Digital Globe
70805,7/16/2018,Digital Globe
70806,5/17/2017,Digital Globe


All of the unknown img_dates are from Bing Maps Arial.
Replace 

In [ ]:
uswtd_df['t_img_date'].fillna('1/1/1900', inplace=True)

In [ ]:
uswtd_df['t_img_date'] = pd.to_datetime(uswtd_df['t_img_date'], format='%m/%d/%Y')

In [ ]:
uswtd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70808 entries, 0 to 70807
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   case_id        70808 non-null  int64         
 1   t_state        70808 non-null  object        
 2   t_county       70808 non-null  object        
 3   t_fips         70808 non-null  int64         
 4   p_name         70808 non-null  object        
 5   p_year         70808 non-null  int64         
 6   p_tnum         70808 non-null  int64         
 7   p_cap          70808 non-null  float64       
 8   t_manu         70808 non-null  object        
 9   t_model        70808 non-null  object        
 10  t_cap          70808 non-null  float64       
 11  t_hh           64628 non-null  float64       
 12  t_rd           64874 non-null  float64       
 13  t_rsa          64874 non-null  float64       
 14  t_ttlh         64628 non-null  float64       
 15  retrofit_year  7080

## t_ttlh

ttlh = (t_rd/2)+t_hh

In [ ]:
uswtd_df[(uswtd_df['t_ttlh'].isna()) & (uswtd_df['t_rd'].notna()) & (uswtd_df['t_hh'].notna())]

,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,loc


no simple calculations are possible to fill in ttlh
First pass, lets just put 0s inplace NaNs

In [ ]:
uswtd_df['t_hh'].fillna(0.0, inplace=True)
uswtd_df['t_rd'].fillna(0.0, inplace=True)
uswtd_df['t_rsa'].fillna(0.0, inplace=True)
uswtd_df['t_ttlh'].fillna(0.0, inplace=True)

In [ ]:
uswtd_df['t_ttlh'].isna().sum()

0

In [ ]:
uswtd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70808 entries, 0 to 70807
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   case_id        70808 non-null  int64         
 1   t_state        70808 non-null  object        
 2   t_county       70808 non-null  object        
 3   t_fips         70808 non-null  int64         
 4   p_name         70808 non-null  object        
 5   p_year         70808 non-null  int64         
 6   p_tnum         70808 non-null  int64         
 7   p_cap          70808 non-null  float64       
 8   t_manu         70808 non-null  object        
 9   t_model        70808 non-null  object        
 10  t_cap          70808 non-null  float64       
 11  t_hh           70808 non-null  float64       
 12  t_rd           70808 non-null  float64       
 13  t_rsa          70808 non-null  float64       
 14  t_ttlh         70808 non-null  float64       
 15  retrofit_year  7080

In [ ]:
if 't_cap' in uswtd_df.columns:
  uswtd_df['t_cap(MW)']=uswtd_df['t_cap']/1000
  #del uswtd_df['t_cap']

In [ ]:
len(uswtd_df.t_fips.unique())

689

# Data visualizations

In [ ]:
gdf = gpd.GeoDataFrame(
    uswtd_df, geometry=gpd.points_from_xy(uswtd_df.xlong, uswtd_df.ylat))
gdf.head()

,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,loc,t_cap(MW),geometry
0,3072661,CA,Kern County,6029,251 Wind,1987,194,18.43,Vestas,Unknown,95.0,0.0,0.0,0.00,0.0,0,2,3,2018-05-08,Digital Globe,-118.363762,35.077908,"(-118.363762, 35.077908)",0.095,POINT (-118.36376 35.07791)
1,3072695,CA,Kern County,6029,251 Wind,1987,194,18.43,Vestas,Unknown,95.0,0.0,0.0,0.00,0.0,0,2,3,2018-05-08,Digital Globe,-118.364410,35.077435,"(-118.36441, 35.077435)",0.095,POINT (-118.36441 35.07744)
2,3072704,CA,Kern County,6029,251 Wind,1987,194,18.43,Vestas,Unknown,95.0,0.0,0.0,0.00,0.0,0,2,3,2018-05-08,Digital Globe,-118.364197,35.077644,"(-118.364197, 35.077644)",0.095,POINT (-118.36420 35.07764)
3,3063272,IA,Story County,19169,30 MW Iowa DG Portfolio,2017,10,30.00,Nordex,AW125/3000,3000.0,87.5,125.0,12271.85,150.0,0,3,3,2017-04-24,Digital Globe,-93.430367,42.028233,"(-93.430367, 42.028233)",3.000,POINT (-93.43037 42.02823)
4,3053390,IA,Boone County,19015,30 MW Iowa DG Portfolio,2017,10,30.00,Nordex,AW125/3000,3000.0,87.5,125.0,12271.85,150.0,0,3,3,2017-06-01,Digital Globe,-93.700424,41.977608,"(-93.700424, 41.977608)",3.000,POINT (-93.70042 41.97761)


In [ ]:
map = folium.Map(location=(37.079284, -98.058082),
                        height=1200, width=1000, 
                        tiles = "Stamen Terrain",
                        zoom_start=8)

gdf_reduced = gdf.drop_duplicates(subset=['p_name'])

fg1 = folium.FeatureGroup(name="Turbine Projects")

for index, row in gdf_reduced.iterrows():

  #  print(row['Location'], row['Centre)ID'])
  fg1.add_child(folium.Marker(location= [row['ylat'], row['xlong']], 
                popup=folium.Popup(f'Project: {row["p_name"]}<br>Year: {row["p_year"]}<br>Project Cap (MW){row["p_cap"]}<br>Num of Turbines: {row["p_tnum"]}', max_width=400)
         ))
map.add_child(fg1)  

map


## Use reduced dataset - only one entry per p_name

In [ ]:
gdf_reduced.head(2)

,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,loc,t_cap(MW),geometry
0,3072661,CA,Kern County,6029,251 Wind,1987,194,18.43,Vestas,Unknown,95.0,0.0,0.0,0.00,0.0,0,2,3,2018-05-08,Digital Globe,-118.363762,35.077908,"(-118.363762, 35.077908)",0.095,POINT (-118.36376 35.07791)
3,3063272,IA,Story County,19169,30 MW Iowa DG Portfolio,2017,10,30.00,Nordex,AW125/3000,3000.0,87.5,125.0,12271.85,150.0,0,3,3,2017-04-24,Digital Globe,-93.430367,42.028233,"(-93.430367, 42.028233)",3.000,POINT (-93.43037 42.02823)


gdf_reduced is where we look 

Drop unknown years from gdf_reduced just to see what things look like

In [ ]:
gdf_reduced_dropped_unknown_years = gdf_reduced[gdf_reduced['p_year']!=0]
gdf_reduced_dropped_unknown_years = gdf_reduced_dropped_unknown_years.groupby(['p_year','t_state']).agg({'p_tnum': 'sum', 'p_cap':'sum'})
gdf_reduced_dropped_unknown_years.reset_index(inplace=True)
gdf_reduced_dropped_unknown_years

,p_year,t_state,p_tnum,p_cap
0,1981,CA,731,0.000
1,1982,CA,928,17.000
2,1983,CA,460,30.000
3,1984,CA,134,8.710
4,1985,CA,1371,130.485
...,...,...,...,...
496,2021,SD,300,615.980
497,2021,TX,1326,1963.980
498,2021,VA,2,12.000
499,2021,WV,4,0.000


In [ ]:
gdf_reduced_dropped_unknown_years.nlargest(n=3, columns=['p_cap'])

,p_year,t_state,p_tnum,p_cap
474,2020,TX,1344,3935.700
452,2019,TX,1501,3933.380
363,2015,TX,1797,3618.878


In [ ]:

px.bar(gdf_reduced_dropped_unknown_years, x='p_year', y='p_cap', color='t_state', title='<B>Renewable Energy Capacity (Wind) by State and Year</B>')

In [ ]:
import plotly.express as px
px.bar(gdf_reduced_dropped_unknown_years, x='p_year', y='p_tnum', color='t_state', title='<B>Number of Turbines by State and Year</B>')

In [ ]:
gdf_reduced_dropped_unknown_years[gdf_reduced_dropped_unknown_years['t_state']=='TX']['p_tnum'].sum()

17762

In [ ]:
tmp_df = gdf_reduced_dropped_unknown_years.drop(columns=['p_year']).groupby(['t_state']).agg({'p_tnum':'sum'}).reset_index()
px.bar(tmp_df, x='t_state', y='p_tnum', title='<B>Turbine Count by State</B>')

In [ ]:
gdf[(gdf['t_state']=='CA') & (gdf['p_year']==2021)]

,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,loc,t_cap(MW),geometry
13122,3110718,CA,Riverside County,6065,Coachella Hills,2021,17,60.6,Vestas,V117-3.6,3600.0,91.5,117.0,10751.32,150.0,0,3,3,2021-04-04,Digital Globe,-116.569199,33.898407,"(-116.569199, 33.898407)",3.60,POINT (-116.56920 33.89841)
13123,3109671,CA,Riverside County,6065,Coachella Hills,2021,17,60.6,Vestas,V117-3.6,3600.0,91.5,117.0,10751.32,150.0,0,3,3,2021-04-04,Digital Globe,-116.569565,33.891884,"(-116.569565, 33.891884)",3.60,POINT (-116.56956 33.89188)
13124,3115250,CA,Riverside County,6065,Coachella Hills,2021,17,60.6,Vestas,V112-3.45,3450.0,94.0,112.0,9852.03,150.0,0,3,1,2021-04-04,Digital Globe,-116.574745,33.898800,"(-116.574745, 33.8988)",3.45,POINT (-116.57474 33.89880)
13125,3115251,CA,Riverside County,6065,Coachella Hills,2021,17,60.6,Vestas,V112-3.45,3450.0,94.0,112.0,9852.03,150.0,0,3,1,2021-04-04,Digital Globe,-116.577431,33.890396,"(-116.577431, 33.890396)",3.45,POINT (-116.57743 33.89040)
13126,3110403,CA,Riverside County,6065,Coachella Hills,2021,17,60.6,Vestas,V117-3.6,3600.0,91.5,117.0,10751.32,150.0,0,3,3,2021-04-04,Digital Globe,-116.577049,33.891804,"(-116.577049, 33.891804)",3.60,POINT (-116.57705 33.89180)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67780,3097390,CA,Kern County,6029,unknown Kern County,2021,41,0.0,Unknown,Unknown,0.0,0.0,0.0,0.00,0.0,0,1,3,2021-02-20,Digital Globe,-118.356224,35.093552,"(-118.356224, 35.093552)",0.00,POINT (-118.35622 35.09355)
67785,3111169,CA,Kern County,6029,unknown Kern County,2021,41,0.0,Unknown,Unknown,0.0,0.0,0.0,0.00,0.0,0,1,3,2021-02-20,Digital Globe,-118.331314,35.092632,"(-118.331314, 35.092632)",0.00,POINT (-118.33131 35.09263)
67786,3097581,CA,Kern County,6029,unknown Kern County,2021,41,0.0,Unknown,Unknown,0.0,0.0,0.0,0.00,0.0,0,1,3,2020-08-06,Digital Globe,-118.366196,35.077084,"(-118.366196, 35.077084)",0.00,POINT (-118.36620 35.07708)
67788,3114207,CA,Kern County,6029,unknown Kern County,2021,41,0.0,Unknown,Unknown,0.0,0.0,0.0,0.00,0.0,0,1,3,2021-08-27,Digital Globe,-118.275948,35.061039,"(-118.275948, 35.061039)",0.00,POINT (-118.27595 35.06104)


## Look at confidence variables
**They are not really corrlated**

In [ ]:
corr_df = gdf[['t_conf_atr', 't_conf_loc']]
corr_df.corr()

,t_conf_atr,t_conf_loc
t_conf_atr,1.000000,0.351443
t_conf_loc,0.351443,1.000000


In [ ]:
tmp_df = gdf[gdf['p_year']>=2021].groupby(['t_state','t_conf_atr']).agg({'p_tnum': 'size'}).reset_index()
px.bar(tmp_df, x='t_state', y='p_tnum', color = 't_conf_atr', title='<B>Confidence Attr by State 2021</B>')

In [ ]:
tmp_df = gdf[gdf['p_year']>=2021].groupby(['t_state','t_conf_loc']).agg({'p_tnum': 'size'}).reset_index()
px.bar(tmp_df, x='t_state', y='p_tnum', color = 't_conf_loc', title='<B>Confidence Location by State 2021</B>')

## Turbine Manufacturers

In [ ]:
tmp_df = gdf.groupby(['t_state', 't_manu']).agg({'p_tnum': 'size'}).reset_index()
px.bar(tmp_df, x='t_state', y='p_tnum', color = 't_manu', title='<B>Turbine Manufacturer by State</B>')

In [ ]:
tmp_df = gdf.groupby(['t_state', 't_manu','p_year']).agg({'p_tnum': 'size'}).reset_index()
px.bar(tmp_df, x='t_state', y='p_tnum', color = 't_manu', title='<B>Turbine Manufacturer by State</B>')

Shorten Changzhou name

In [ ]:
gdf[gdf.t_manu.str.startswith('Changzhou')]
gdf.loc[54095,'t_manu'] = 'Changzhou'
gdf.loc[[54095]]

,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,loc,t_cap(MW),geometry
54095,3031201,MN,Scott County,27139,Shakopee Mdewakanton Sioux,2009,1,1.5,Changzhou,FD-77-1500,1500.0,80.0,77.0,4656.63,118.6,0,3,3,2019-12-10,Digital Globe,-93.482994,44.736092,"(-93.482994, 44.736092)",1.5,POINT (-93.48299 44.73609)


In [ ]:
gdf['t_manu'] = gdf.t_manu.str.replace(r'(Siemens G.*$)','Siemens')
gdf[gdf.t_manu.str.startswith('Siemens')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

The default value of regex will change from True to False in a future version.



,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,loc,t_cap(MW),geometry
25,3025854,IA,Adair County,19001,Adair,2008,76,174.8,Siemens,SWT-2.3-93,2300.0,80.0,93.0,6792.91,126.5,0,2,3,2018-10-28,Digital Globe,-94.638390,41.439693,"(-94.63839, 41.439693)",2.3,POINT (-94.63839 41.43969)
26,3025816,IA,Cass County,19029,Adair,2008,76,174.8,Siemens,SWT-2.3-93,2300.0,80.0,93.0,6792.91,126.5,0,2,3,2019-10-31,Digital Globe,-94.712090,41.468391,"(-94.71209, 41.468391)",2.3,POINT (-94.71209 41.46839)
27,3025812,IA,Adair County,19001,Adair,2008,76,174.8,Siemens,SWT-2.3-93,2300.0,80.0,93.0,6792.91,126.5,0,2,3,2018-10-28,Digital Globe,-94.630989,41.481194,"(-94.630989, 41.481194)",2.3,POINT (-94.63099 41.48119)
28,3025863,IA,Adair County,19001,Adair,2008,76,174.8,Siemens,SWT-2.3-93,2300.0,80.0,93.0,6792.91,126.5,0,2,3,2019-10-31,Digital Globe,-94.688995,41.435894,"(-94.688995, 41.435894)",2.3,POINT (-94.68900 41.43589)
29,3025858,IA,Adair County,19001,Adair,2008,76,174.8,Siemens,SWT-2.3-93,2300.0,80.0,93.0,6792.91,126.5,0,2,3,2018-10-28,Digital Globe,-94.618294,41.441593,"(-94.618294, 41.441593)",2.3,POINT (-94.61829 41.44159)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65837,3033004,WA,Klickitat County,53039,Windy Point IIa (Windy Flats Extension),2009,26,59.8,Siemens,SWT-2.3-93,2300.0,80.0,93.0,6792.91,126.5,0,3,3,2011-01-01,NAIP,-120.846588,45.705692,"(-120.846588, 45.705692)",2.3,POINT (-120.84659 45.70569)
65838,3033014,WA,Klickitat County,53039,Windy Point IIa (Windy Flats Extension),2009,26,59.8,Siemens,SWT-2.3-93,2300.0,80.0,93.0,6792.91,126.5,0,3,3,2011-01-01,NAIP,-120.882088,45.701195,"(-120.882088, 45.701195)",2.3,POINT (-120.88209 45.70119)
65839,3032998,WA,Klickitat County,53039,Windy Point IIa (Windy Flats Extension),2009,26,59.8,Siemens,SWT-2.3-93,2300.0,80.0,93.0,6792.91,126.5,0,3,3,2011-01-01,NAIP,-120.862885,45.708294,"(-120.862885, 45.708294)",2.3,POINT (-120.86289 45.70829)
65840,3033000,WA,Klickitat County,53039,Windy Point IIa (Windy Flats Extension),2009,26,59.8,Siemens,SWT-2.3-93,2300.0,80.0,93.0,6792.91,126.5,0,3,3,2011-01-01,NAIP,-120.838585,45.714794,"(-120.838585, 45.714794)",2.3,POINT (-120.83858 45.71479)


## Turbines by State

In [ ]:
tmp_df = gdf.groupby(['t_state','t_manu']).agg({'p_tnum': 'size'}).reset_index()
#tmp_df = tmp_df[tmp_df['p_year']>2010]
#tmp_df = tmp_df.sort_values('p_year', ascending=True)
#tmp_df1 = tmp_df.sort_values(by=['p_year'])

#px.scatter(tmp_df1, size_max = 60, x='t_state',animation_frame='p_year', y='t_manu', size='p_tnum', color = 't_manu', title='<B>Turbine Manufacturer by State</B>')
px.scatter(tmp_df, size_max = 70, x='t_state', y='t_manu', size='p_tnum', color = 't_manu', title='<B>Turbine Manufacturer by State</B>')

In [ ]:
tmp_df

,t_state,t_manu,p_year,p_tnum
4,AK,EWT,2012,2
5,AK,EWT,2013,3
6,AK,EWT,2018,1
7,AK,EWT,2019,1
11,AK,GE Wind,2012,14
...,...,...,...,...
1112,WY,GE Wind,2016,46
1113,WY,GE Wind,2020,306
1123,WY,Unknown,2020,97
1129,WY,Vestas,2020,95


In [ ]:
tmp_df = gdf.groupby(['t_state','t_manu','p_year']).agg({'p_tnum': 'size'}).reset_index()
tmp_df = tmp_df[tmp_df['p_year']>2010]
tmp_df = tmp_df[(tmp_df['t_state']=='TX') | (tmp_df['t_state']=='NE') |(tmp_df['t_state']=='CA')]
#tmp_df = tmp_df.sort_values('p_year', ascending=True)
tmp_df1 = tmp_df.sort_values(by=['p_year'])

px.scatter(tmp_df1, size_max=60, x='t_state',animation_frame='p_year', y='t_manu', size='p_tnum', color = 't_manu', title='<B>Turbine Manufacturer by State</B>')

# Read lightening data

Import the lightning data - in excel, I added 2 letter state code for easy lookup

In [ ]:
lightning_by_state_df.rename(columns={'StateCode': 't_state'}, inplace=True)
lightning_by_state_df

,t_state,State,Total Lightning in 2019,Average Total Lightning 2015 to 2019,Average Lightning Count Per Square Mile 2015 to 2019
0,AL,Alabama,3970233,5773460,110.7
1,AK,Alaska,383291,178190,0.3
2,AZ,Arizona,1841300,3218418,28.2
3,AR,Arkansas,7442101,7751123,145.6
4,CA,California,432207,573698,3.6
5,CO,Colorado,3704799,3624165,34.8
6,CT,Connecticut,87830,75534,15.3
7,DE,Delaware,180209,158376,79.8
8,DC,D.C.,8005,7195,106.0
9,FL,Florida,13049687,13989300,244.1


In [ ]:
combined_df = pd.merge(gdf_reduced, lightning_by_state_df, 
                      on ='t_state', 
                      how ='inner')

In [ ]:
combined_df

,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,loc,t_cap(MW),geometry,State,Total Lightning in 2019,Average Total Lightning 2015 to 2019,Average Lightning Count Per Square Mile 2015 to 2019
0,3072661,CA,Kern County,6029,251 Wind,1987,194,18.43,Vestas,Unknown,95.0,0.0,NaN,NaN,0.0,0,2,3,2018-05-08,Digital Globe,-118.363762,35.077908,"(-118.363762, 35.077908)",0.095,POINT (-118.36376 35.07791),California,432207,573698,3.6
1,3025447,CA,Kern County,6029,Alite Wind Farm,2008,8,24.00,Vestas,V90-3.0,3000.0,80.0,90.0,6361.73,125.0,0,3,3,2020-01-16,Digital Globe,-118.344086,35.035896,"(-118.344086, 35.035896)",3.000,POINT (-118.34409 35.03590),California,432207,573698,3.6
2,3033111,CA,Kern County,6029,Alta I,2010,100,150.00,GE Wind,GE1.5-77,1500.0,80.0,77.0,4656.63,118.6,0,2,3,2012-01-01,NAIP,-118.370987,35.027596,"(-118.370987, 35.027596)",1.500,POINT (-118.37099 35.02760),California,432207,573698,3.6
3,3048645,CA,Kern County,6029,Alta II,2010,50,150.00,Vestas,V90-3.0,3000.0,80.0,90.0,6361.73,125.0,0,3,3,2012-01-01,NAIP,-118.257988,35.028294,"(-118.257988, 35.028294)",3.000,POINT (-118.25799 35.02829),California,432207,573698,3.6
4,3081835,CA,Kern County,6029,Alta III,2011,50,150.00,Vestas,V90-3.0,3000.0,80.0,90.0,6361.73,125.0,0,3,3,2016-11-09,Digital Globe,-118.233887,35.009499,"(-118.233887, 35.009499)",3.000,POINT (-118.23389 35.00950),California,432207,573698,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,3041065,NV,White Pine County,32033,Spring Valley,2012,66,151.80,Siemens,SWT-2.3-101,2300.0,80.0,101.0,8011.85,130.5,0,3,3,2021-01-30,Digital Globe,-114.475685,39.107094,"(-114.475685, 39.107094)",2.300,POINT (-114.47568 39.10709),Nevada,721732,819506,7.4
1698,3001376,NV,Clark County,32003,unknown Clark County,2014,1,0.00,Unknown,Unknown,0.0,0.0,NaN,NaN,0.0,0,1,3,1900-01-01,Bing Maps Aerial,-114.836189,35.955196,"(-114.836189, 35.955196)",0.000,POINT (-114.83619 35.95520),Nevada,721732,819506,7.4
1699,3039872,NV,Lander County,32015,unknown Lander County,2012,1,0.00,Unknown,Unknown,0.0,0.0,NaN,NaN,0.0,0,1,3,2017-07-15,Digital Globe,-117.268288,39.431965,"(-117.268288, 39.431965)",0.000,POINT (-117.26829 39.43196),Nevada,721732,819506,7.4
1700,3029663,DE,Sussex County,10005,University of Delaware,2010,1,2.00,Gamesa,G90-2.0,2000.0,78.0,90.0,6361.73,123.1,0,3,3,1900-01-01,Bing Maps Aerial,-75.164902,38.782890,"(-75.164902, 38.78289)",2.000,POINT (-75.16490 38.78289),Delaware,180209,158376,79.8


In [ ]:
c_df = combined_df.groupby(['t_state']).agg({'p_tnum': 'sum', 'Average Lightning Count Per Square Mile 2015 to 2019': 'mean'}).reset_index()

In [ ]:
c_df

,t_state,p_tnum,Average Lightning Count Per Square Mile 2015 to 2019
0,AK,156,0.3
1,AR,1,145.6
2,AZ,270,28.2
3,CA,7951,3.6
4,CO,3049,34.8
5,CT,3,15.3
6,DE,1,79.8
7,FL,1,244.1
8,HI,132,3.2
9,IA,5835,95.7


## Plot Turbine Manufacturer by State and Lightning strikes

In [ ]:
px.scatter(c_df, size_max = 60, x='t_state', size = 'p_tnum', color='t_state', y = 'Average Lightning Count Per Square Mile 2015 to 2019', title='<B>Turbine Manufacturer by State - Size indicates number of turbines deployed</B>')